In [ ]:
import pandas_datareader.data as web
from pymatgen import Structure, MPRester
import plotly.express as px

from mpcontribs.client import load_client
from mpcontribs.io.core.components.hdata import HierarchicalData
from mpcontribs.io.core.components.tdata import Table
from mpcontribs.io.core.components.gdata import Plot

Use your API key to set up the client.

In [ ]:
project = 'my_project'
headers = {'X-Consumer-Username': 'phuck@lbl.gov'}
client = load_client(headers=headers)
# client = load_client('your-api-key-here')

The *dir* command lists available resources and operations.

In [ ]:
print(dir(client))
print(dir(client.projects))

Use the *projects* resource to retrieve or update project information. The *_fields* argument controls which fields are included in the response.

In [ ]:
HierarchicalData(client.projects.get_entry(
    pk=project, _fields=['title', 'authors', 'description', 'urls']
).response().result)

The *update_entry* operation returns a dictionary of updated fields.

In [ ]:
client.projects.update_entry(pk=project, project={
     'title': 'New Title',
#     'urls': {'PRC': 'https://google.fr'},
#     'urls': {'PRC': None}
}).response().result

Use the *contributions* resource to create, retrieve, and update contributions.

In [ ]:
identifier = 'mp-1002'
cid = client.contributions.create_entry(contribution={
    'project': project, 'identifier': identifier,
    'data': {'formula': 'CO2', 'E': '3.33 eV', 'E|V': {'a': 1, 'b': '3 cm'}}
}).response().result['id']

In [ ]:
HierarchicalData(client.contributions.get_entries(
    project=project, _fields=['id', 'identifier', 'data']
).response().result)

In [ ]:
client.contributions.update_entry(pk=cid, contribution={
        'data': {'formula': 'H2', 'E': '14 eV'},
        'identifier': 'mp-1004'
}).response().result

Use the *tables* resource to add tables to contributions.

In [ ]:
table = Table(web.DataReader('^DJI', 'stooq'), contribution=cid, name='Dow-Jones Index')

In [ ]:
tid = client.tables.create_entry(table=table.to_dict()).response().result['id']

In [ ]:
entry = client.tables.get_entry(
    pk=tid, _fields=['columns', 'data'], data_per_page=3, data_page=5
).response().result
Table.from_dict(entry)

In [ ]:
Plot.from_dict(entry)

In [ ]:
client.tables.update_entry(pk=tid, table={'name': 'DJI'}).response().result

Use the *structures* resource to add structures to contributions.

In [ ]:
with MPRester() as m:
    structure = m.get_structure_by_material_id("mp-1234")

In [ ]:
sdct = dict(contribution=cid, name='LuAl2')
sdct.update(structure.as_dict())
sid = client.structures.create_entry(structure=sdct).response().result['id']

In [ ]:
Structure.from_dict(client.structures.get_entry(pk=sid, _fields=['lattice', 'sites']).response().result)

In [ ]:
client.structures.update_entry(pk=sid, structure={'name': 'Lu Al2'}).response().result

Finally, publish and/or delete the project or its components.

In [ ]:
# TODO use update_entries to also publish according contributions, tables, and structures

In [ ]:
client.structures.delete_entry(pk=sid).response().result
client.tables.delete_entry(pk=tid).response().result

deleting contributions (projects) also deletes corresponding structures, tables, notebooks and cards (contributions).

In [ ]:
client.contributions.delete_entry(pk=cid).response().result  
# client.projects.delete_entry(pk=project).response().result

**TODO** how to add a custom overview graph